# Using BGE M3-Embedding Model with Milvus

As Deep Neural Networks continue to advance rapidly, it's increasingly common to employ them for information representation and retrieval. Referred to as embedding models, they can encode information into dense or sparse vector representations within a multi-dimensional space.


On January 30, 2024, a new member called BGE-M3 was released as part of the BGE model series. The M3 represents its capabilities in supporting over 100 languages, accommodating input lengths of up to 8192, and incorporating multiple functions such as dense, lexical, and multi-vec/colbert retrieval into a unified system. BGE-M3 holds the distinction of being the first embedding model to offer support for all three retrieval methods, resulting in achieving state-of-the-art performance on multi-lingual (MIRACL) and cross-lingual (MKQA) benchmark tests.

Milvus, world's first open-source vector database, plays a vital role in semantic search with efficient storage and retrieval for vector embeddings. Its scalability and advanced functionalities, such as metadata filtering, further contribute to its significance in this field. 

This tutorial shows how to use **BGE M3 embedding model with Milvus** for semantic similarity search.

![](../../images/bge_m3.png)


## Preparations

We will demonstrate with `BAAI/bge-m3` model and Milvus in Standalone mode. The text for searching comes from the [M3 paper](https://arxiv.org/pdf/2402.03216.pdf). For each sentence in the paper, we use `BAAI/bge-m3` model to convert the text string into 1024 dimension vector embedding, and store each embedding in Milvus.

We then search a query by converting the query text into a vector embedding, and perform vector Approximate Nearest Neighbor search to find the text strings with cloest semantic.

To run this demo, be sure you have already [started up a Milvus instance](https://milvus.io/docs/install_standalone-docker.md) and installed python packages `pymilvus` (Milvus client library) and `FlagEmbedding` (library for BGE models).

In [ ]:
! pip install pymilvus FlagEmbedding

Import packages.

In [1]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)
from FlagEmbedding import BGEM3FlagModel


Set up the options for Milvus, specify model name as `BAAI/bge-m3`.

In [2]:
MILVUS_HOST = "localhost"
MILVUS_PORT = "19530"
COLLECTION_NAME = "bge_m3_doc_collection"  # Milvus collection name
EMBEDDING_MODEL = "BAAI/bge-m3"

Let’s try the BGE M3 Embedding service with a text string, print the result vector embedding and get the dimensions of the model.

In [3]:
test_sentences = "What is BGE M3?"

model = BGEM3FlagModel(EMBEDDING_MODEL, use_fp16=True)  # Setting use_fp16 to True speeds up computation with a slight performance degradation


test_embedding = model.encode([test_sentences])['dense_vecs'][0]

print(f'{test_embedding[:20]} ...')
dimension = len(test_embedding)
print(f'\nDimensions of `{EMBEDDING_MODEL}` embedding model is: {dimension}')

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

loading existing colbert_linear and sparse_linear---------
----------using 4*GPUs----------
[-0.03415   -0.04712   -0.0009007 -0.04697    0.04025   -0.07654
 -0.001877   0.007637  -0.01312   -0.007435  -0.0712     0.0526
  0.02162   -0.04178    0.000628  -0.05307    0.00796   -0.0431
  0.01224   -0.006145 ] ...

Dimensions of `BAAI/bge-m3` embedding model is: 1024


## Load vectors to Milvus

We need creat a collection in Milvus and build index so that we can efficiently search vectors. For more information on how to use Milvus, check out the [documentation](https://milvus.io/docs/example_code.md).


In [4]:
# Connect to Milvus
connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)

# Remove collection if it already exists
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

# Set scheme with 3 fields: id (int), text (string), and embedding (float array).
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65_535),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dimension)
]
schema = CollectionSchema(fields, "Here is description of this collection.")
# Create a collection with above schema.
doc_collection = Collection(COLLECTION_NAME, schema)

# Create an index for the collection.
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
doc_collection.create_index("embedding", index)

Status(code=0, message=)

Here we have prepared a data set of text strings from the [M3 paper](https://arxiv.org/pdf/2402.03216.pdf), named `m3_paper.txt`. It stores each sentence as a line, and we convert each line in the document into a dense vector embedding with `BAAI/bge-m3` and then insert these embeddings into Milvus collection.

In [5]:
with open('./docs/m3_paper.txt', 'r') as f:
    lines = f.readlines()

embeddings = model.encode(lines)['dense_vecs']
entities = [
    list(range(len(lines))),  # field id (primary key) 
    lines,  # field text
    embeddings,  #field embedding
]
insert_result = doc_collection.insert(entities)

# In Milvus, it's a best practice to call flush() after all vectors are inserted,
# so that a more efficient index is built for the just inserted vectors.
doc_collection.flush()

## Query

Here we will build a `semantic_search` function, which is used to retrieve the topK most semantically similar document from a Milvus collection.


In [6]:
# Load the collection into memory for searching
doc_collection.load()


def semantic_search(query, top_k=3):
    vectors_to_search = model.encode([query])['dense_vecs']
    search_params = {
        "metric_type": "L2",
        "params": {"nprobe": 10},
    }
    result = doc_collection.search(vectors_to_search, "embedding", search_params, limit=top_k, output_fields=["text"])
    return result[0]

Here we ask a question about the embedding models.

In [13]:
question = 'How many working languages does the M3-Embedding model support?'

match_results = semantic_search(question, top_k=3)
for match in match_results:
    print(f"distance = {match.distance:.2f}\n{match.entity.text}")

distance = 0.46
Particularly, M3-Embedding is proficient in multilinguality, which is able to support more than 100 world languages.

distance = 0.53
1) We present M3-Embedding, which is the first model which supports multi-linguality, multifunctionality, and multi-granularity.

distance = 0.63
In this paper, we present M3-Embedding, which achieves notable versatility in supporting multilingual retrieval, handling input of diverse granularities, and unifying different retrieval functionalities.



The smaller the distance, the closer the vector is, that is, semantically more similar. We can see that the top 1 result returned *"M3-Embedding...more than 100 world languages..."* can directly answer the question.

Let's try another question.

In [23]:
question = 'What are the sources of data used in the training dataset?'

match_results = semantic_search(question, top_k=3)
for match in match_results:
    print(f"distance = {match.distance:.2f}\n{match.entity.text}")

distance = 0.61
The three data sources complement to each other, which are applied to different stages of the training process.

distance = 0.69
Our dataset consists of three sources: 1) the extraction of unsupervised data from massive multi-lingual corpora, 2) the integration of closely related supervised data, 3) the synthesization of scarce training data.

distance = 0.74
In this   Table 1: Specification of training data.



In this example, the top 2 results have enough information to answer the question. By selecting the top K results, semantic search with embedding model and vector retrieval is able to identify the meaning of queries and return the most semantically similar documents. Plugging this solution with Large Language Model (a pattern referred to as Retrieval Augmented Generation), a more human-readable answer can be crafted.

We can delete this collection to save resources.

In [24]:
# Drops the collection
utility.drop_collection(COLLECTION_NAME)

In this notebook, we showed how to generate dense vectors with BGE M3 embedding model and use Milvus to perform semantic search. In the upcoming releases, Milvus will support hybrid search with dense and sparse vectors, which BGE M3 model can produce at the same time.

Milvus has integrated with all major model providers, including OpenAI, HuggingFace and many more. You can learn about Milvus at https://milvus.io/docs.